# DMT Creep Protocol: Viscosity Bifurcation and Delayed Yielding

## Learning Objectives

1. **Maxwell Elastic Jump**: Understand instantaneous elastic response γ_e(0) = σ₀/G in creep
2. **Viscosity Bifurcation**: Distinguish sub-yield (bounded strain) vs supra-yield (unbounded) regimes
3. **Delayed Yielding**: Observe thixotropic delayed flow onset near critical stress
4. **Real Data Fitting**: Fit biological mucus creep compliance data

## Prerequisites

- Understanding of creep compliance J(t) = γ(t)/σ₀
- Familiarity with DMT structural kinetics (λ parameter)
- Knowledge of Maxwell viscoelastic model

## Runtime

- NLSQ fitting: ~5-15 seconds
- Bayesian inference: ~2-5 minutes (1000 warmup + 2000 samples, 4 chains)
- Total: ~5-10 minutes

## Setup

In [ ]:
# Google Colab setup
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax arviz
    import os
    os.environ['JAX_ENABLE_X64'] = '1'
except ImportError:
    IN_COLAB = False

In [ ]:
# Core imports
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit
from IPython.display import display

# JAX-safe imports
from rheojax.core.jax_config import safe_import_jax, verify_float64
jax, jnp = safe_import_jax()
verify_float64()

# RheoJAX imports
from rheojax.models import DMTLocal
from rheojax.core.parameters import Parameter, ParameterSet

# Bayesian imports
import arviz as az

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit,
    display_arviz_diagnostics,
    plot_posterior_predictive,
)

FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"Float64 enabled: {jax.config.jax_enable_x64}")
print(f"FAST_MODE: {FAST_MODE}")

## Theory: DMT Creep Protocol

### Creep Test

Apply constant stress σ₀ and measure strain γ(t) evolution:

**Creep compliance**: J(t) = γ(t) / σ₀

### Maxwell Variant Response

For DMT with Maxwell backbone (`include_elasticity=True`):

1. **Elastic jump** (instantaneous): γ_e(0) = σ₀ / G
2. **Viscous flow**: γ̇_v = σ₀ / η(λ) where η(λ) depends on structure
3. **Structure evolution**: dλ/dt = (1-λ)/t_eq - aλ|γ̇_v|^c / t_eq

### Viscosity Bifurcation

The exponential closure η = η_∞(η_0/η_∞)^λ creates two regimes:

- **Sub-yield** (σ₀ < σ_y): Structure rebuilds, viscosity increases, flow stops → bounded γ(t)
- **Supra-yield** (σ₀ > σ_y): Structure breaks down, viscosity decreases, continuous flow → unbounded γ(t)

### Physical Interpretation

- Near critical stress: **delayed yielding** due to slow structure evolution
- Biological relevance: mucus flow under gravity vs applied pressure
- Characteristic timescale: t_eq (structure equilibration time)

## Load Real Creep Data: Biological Mucus

In [ ]:
# Load mucus creep compliance data
data_path = os.path.join("..", "data", "creep", "biological", "creep_mucus_data.csv")

if not os.path.exists(data_path):
    # Fallback to alternative path structures
    alt_paths = [
        "../data/creep/biological/creep_mucus_data.csv",
        "data/creep/biological/creep_mucus_data.csv",
        "examples/data/creep/biological/creep_mucus_data.csv"
    ]
    for path in alt_paths:
        if os.path.exists(path):
            data_path = path
            break
    else:
        raise FileNotFoundError(
            f"Creep data not found. Expected at: {data_path}\n"
            "Please ensure the data file exists or adjust the path."
        )

# Load data (tab-separated, with header)
raw = np.loadtxt(data_path, delimiter="\t", skiprows=1)
time_data = raw[:, 0]  # Time [s]
J_data = raw[:, 1]     # Creep compliance [1/Pa]

print(f"Loaded {len(time_data)} data points")
print(f"Time range: {time_data[0]:.2e} - {time_data[-1]:.2e} s")
print(f"Compliance range: {J_data[0]:.2e} - {J_data[-1]:.2e} 1/Pa")

### Data Inspection and Conversion

**Important**: The data represents creep compliance J(t) = γ(t) / σ₀, not strain directly.

To convert to strain: γ(t) = J(t) × σ₀

For fitting purposes, we assume σ₀ = 1 Pa, so numerically γ(t) ≈ J(t).

In [ ]:
# Assume applied stress for conversion
sigma_0 = 1.0  # Pa (assumed - if σ₀=1, then γ=J)

# Convert compliance to strain
gamma_data = J_data * sigma_0

# Plot raw data
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
axes[0].plot(time_data, J_data, 'o', markersize=4, alpha=0.6, label='Experimental')
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Creep Compliance J(t) [1/Pa]')
axes[0].set_title('Mucus Creep Compliance (Linear Scale)')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Log-log scale
axes[1].loglog(time_data, J_data, 'o', markersize=4, alpha=0.6, label='Experimental')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Creep Compliance J(t) [1/Pa]')
axes[1].set_title('Mucus Creep Compliance (Log-Log Scale)')
axes[1].grid(True, alpha=0.3, which='both')
axes[1].legend()

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nAssumed applied stress: σ₀ = {sigma_0} Pa")
print(f"Strain range: {gamma_data[0]:.2e} - {gamma_data[-1]:.2e}")

## NLSQ Fitting: Fast Parameter Estimation

We use NLSQ optimization to fit the DMT creep model to data.

**Strategy**:
1. Create wrapper function that calls `model.simulate_creep()`
2. Interpolate simulation output to match data time points
3. Use NLSQ for fast fitting (5-270x faster than scipy)

In [ ]:
# Initialize model with Maxwell elasticity
model = DMTLocal(closure="exponential", include_elasticity=True)

# Create ParameterSet for fitting using DMT model parameter names
from rheojax.core.parameters import ParameterSet

params = ParameterSet()
params.add('eta_inf', value=1e2, bounds=(1e0, 1e6))      # Low-structure viscosity [Pa·s]
params.add('eta_0', value=1e5, bounds=(1e2, 1e8))        # High-structure viscosity [Pa·s]
params.add('t_eq', value=10.0, bounds=(0.1, 1e3))        # Equilibration time [s]
params.add('a', value=1.0, bounds=(0.1, 10.0))           # Structure breakdown rate
params.add('c', value=1.0, bounds=(0.5, 2.0))            # Shear-rate exponent
params.add('G0', value=100.0, bounds=(1.0, 1e4))         # Elastic modulus [Pa]
params.add('m_G', value=1.0, bounds=(0.1, 3.0))          # Structure exponent for G

print("Initial parameters:")
for name in params.keys():
    p = params[name]
    print(f"  {name}: {p.value:.2e} (bounds: {p.bounds})")

In [ ]:
# Create scipy-compatible wrapper function
def dmt_creep_fn(t_data, eta_inf, eta_0, t_eq, a, c, G0, m_G):
    """
    Wrapper for DMT creep simulation using scipy-compatible signature.
    
    Args:
        t_data: Time points for evaluation
        eta_inf, eta_0, t_eq, a, c, G0, m_G: Model parameters
    
    Returns:
        Predicted strain values at t_data points
    """
    # Set model parameters
    model.parameters.set_value("eta_inf", float(eta_inf))
    model.parameters.set_value("eta_0", float(eta_0))
    model.parameters.set_value("t_eq", float(t_eq))
    model.parameters.set_value("a", float(a))
    model.parameters.set_value("c", float(c))
    model.parameters.set_value("G0", float(G0))
    model.parameters.set_value("m_G", float(m_G))
    
    # Simulate creep (returns time, strain, strain_rate, lambda)
    t_end = float(np.max(t_data)) * 1.2  # Simulate slightly beyond last data point
    dt = t_end / 500  # Use 500 points
    t_sim, gamma_sim, gamma_dot_sim, lambda_sim = model.simulate_creep(
        sigma_0=sigma_0,
        t_end=t_end,
        dt=dt
    )
    
    # Interpolate to data time points
    gamma_pred = np.interp(t_data, np.array(t_sim), np.array(gamma_sim))
    
    return gamma_pred

print("Created scipy-compatible wrapper function for DMT creep simulation")

In [ ]:
# Perform optimization using scipy.optimize.curve_fit
print("Starting optimization with scipy.optimize.curve_fit...")

# Helper function for computing fit quality
def compute_fit_quality(y_true, y_pred):
    """Compute R² and RMSE."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    residuals = y_true - y_pred
    if y_true.ndim > 1:
        residuals = residuals.ravel()
        y_true = y_true.ravel()
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    rmse = np.sqrt(np.mean(residuals**2))
    return {"R2": r2, "RMSE": rmse}

# Initial guesses - eta_inf must be < 100 (model constraint)
p0 = [10.0, 1e5, 10.0, 1.0, 1.0, 100.0, 1.0]

# Bounds (lower, upper) - must match DMT model constraints
# eta_inf: (0.001, 100.0), eta_0: (100.0, 1e8), t_eq: (0.1, 10000.0)
# a: (0.001, 100.0), c: (0.1, 2.0), G0: (1.0, 1e6), m_G: (0.5, 2.0)
bounds = (
    [0.001, 1e2, 0.1, 0.1, 0.5, 1.0, 0.5],    # lower
    [99.9, 1e8, 1e4, 10.0, 2.0, 1e4, 2.0]     # upper (eta_inf < 100 to avoid bound violation)
)

popt, pcov = curve_fit(
    dmt_creep_fn,
    time_data,
    gamma_data,
    p0=p0,
    bounds=bounds,
    maxfev=5000
)

# Parameter names
param_names = ['eta_inf', 'eta_0', 't_eq', 'a', 'c', 'G0', 'm_G']

# Compute predictions and metrics
gamma_pred = dmt_creep_fn(time_data, *popt)
metrics = compute_fit_quality(gamma_data, gamma_pred)

print(f"\nOptimization converged successfully")
print(f"R² score: {metrics['R2']:.6f}")
print(f"RMSE: {metrics['RMSE']:.6e}")
print(f"\nFitted parameters:")

fitted_params = {}
for name, value in zip(param_names, popt):
    fitted_params[name] = float(value)
    print(f"  {name}: {value:.6e}")

# Update model with fitted parameters
for name, value in fitted_params.items():
    model.parameters.set_value(name, value)

In [ ]:
# Generate fitted curve
t_end_fit = time_data[-1] * 1.1
dt_fit = t_end_fit / 500
t_fit, gamma_fit, gamma_dot_fit, lambda_fit = model.simulate_creep(
    sigma_0=sigma_0,
    t_end=t_end_fit,
    dt=dt_fit
)

# Plot fit vs data
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Strain - Linear scale
axes[0, 0].plot(time_data, gamma_data, 'o', markersize=4, alpha=0.6, label='Data')
axes[0, 0].plot(t_fit, gamma_fit, '-', linewidth=2, label=f'DMT Fit (R²={metrics["R2"]:.4f})')
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Strain γ(t)')
axes[0, 0].set_title('Creep Response (Linear Scale)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Strain - Log-log scale
axes[0, 1].loglog(time_data, gamma_data, 'o', markersize=4, alpha=0.6, label='Data')
axes[0, 1].loglog(t_fit, gamma_fit, '-', linewidth=2, label='DMT Fit')
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Strain γ(t)')
axes[0, 1].set_title('Creep Response (Log-Log Scale)')
axes[0, 1].grid(True, alpha=0.3, which='both')
axes[0, 1].legend()

# Strain rate evolution
axes[1, 0].semilogy(t_fit, jnp.abs(gamma_dot_fit), '-', linewidth=2, color='C2')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Strain Rate |γ̇(t)| [1/s]')
axes[1, 0].set_title('Strain Rate Evolution')
axes[1, 0].grid(True, alpha=0.3)

# Structure parameter evolution
axes[1, 1].plot(t_fit, lambda_fit, '-', linewidth=2, color='C3')
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Structure Parameter λ(t)')
axes[1, 1].set_title('Structure Evolution')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim([0, 1.1])

plt.tight_layout()
display(fig)
plt.close(fig)

## Bayesian Inference: Uncertainty Quantification

Use NUTS sampling to:
1. Quantify parameter uncertainty
2. Check parameter correlations
3. Generate posterior predictive distributions

In [ ]:
# Set model attributes for Bayesian inference
model._sigma_applied = sigma_0
model._creep_lam_init = 1.0  # Assume fully structured initial state

print("Starting Bayesian inference (NUTS)...")
print("This will take 2-5 minutes with 4 chains...")

# Bayesian inference with NLSQ warm-start
posterior_result = model.fit_bayesian(
    time_data,
    gamma_data,
    test_mode='creep',
    num_warmup=1000,
    num_samples=2000,
    num_chains=4,
    seed=42
)

print("\nBayesian inference complete!")

In [ ]:
# Compute diagnostics
samples = posterior_result.posterior_samples

# R-hat convergence diagnostic
print("\nConvergence Diagnostics (R-hat):")
for param_name in param_names:
    if param_name in samples:
        param_samples = samples[param_name]
        r_hat = az.rhat(param_samples)
        print(f"  {param_name}: {float(r_hat):.4f}")

# Effective sample size
print("\nEffective Sample Size (ESS):")
for param_name in param_names:
    if param_name in samples:
        param_samples = samples[param_name]
        ess = az.ess(param_samples)
        print(f"  {param_name}: {float(ess):.0f}")

# Credible intervals
print("\n95% Credible Intervals:")
intervals = model.get_credible_intervals(samples, credibility=0.95)
for param_name in param_names:
    if param_name in intervals:
        lower, upper = intervals[param_name]
        median = jnp.median(samples[param_name])
        print(f"  {param_name}: [{lower:.6e}, {upper:.6e}], median: {median:.6e}")

In [ ]:
display_arviz_diagnostics(posterior_result, param_names, fast_mode=FAST_MODE)

In [ ]:
# Posterior predictive check
n_posterior_samples = 100
posterior_predictions = []

# Sample from posterior
for i in range(n_posterior_samples):
    # Random sample from flattened posterior
    idx = np.random.randint(0, samples[param_names[0]].size)
    
    # Get individual parameter values
    sample_eta_inf = float(samples['eta_inf'].flatten()[idx])
    sample_eta_0 = float(samples['eta_0'].flatten()[idx])
    sample_t_eq = float(samples['t_eq'].flatten()[idx])
    sample_a = float(samples['a'].flatten()[idx])
    sample_c = float(samples['c'].flatten()[idx])
    sample_G0 = float(samples['G0'].flatten()[idx])
    sample_m_G = float(samples['m_G'].flatten()[idx])
    
    # Predict using scipy-style wrapper
    gamma_pred_sample = dmt_creep_fn(
        time_data, sample_eta_inf, sample_eta_0, sample_t_eq, 
        sample_a, sample_c, sample_G0, sample_m_G
    )
    posterior_predictions.append(gamma_pred_sample)

posterior_predictions = jnp.array(posterior_predictions)

# Compute credible interval
lower_bound = jnp.percentile(posterior_predictions, 2.5, axis=0)
upper_bound = jnp.percentile(posterior_predictions, 97.5, axis=0)
median_pred = jnp.median(posterior_predictions, axis=0)

# Plot posterior predictive
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(time_data, gamma_data, 'o', markersize=4, alpha=0.6, label='Data', zorder=3)
ax.plot(time_data, median_pred, '-', linewidth=2, color='C1', label='Posterior Median', zorder=2)
ax.fill_between(time_data, lower_bound, upper_bound, alpha=0.3, color='C1', 
                 label='95% Credible Interval', zorder=1)

ax.set_xlabel('Time [s]')
ax.set_ylabel('Strain γ(t)')
ax.set_title('Posterior Predictive Check')
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
display(fig)
plt.close(fig)

## Physics: Viscosity Bifurcation Analysis

Now we explore how the DMT model predicts different behavior at different stress levels:

1. **Sub-yield regime**: Structure rebuilds faster than breakdown → bounded strain
2. **Critical regime**: Balance between rebuilding and breakdown → delayed yielding
3. **Supra-yield regime**: Breakdown dominates → unbounded flow

In [ ]:
# Test multiple stress levels
stress_levels = jnp.array([0.5, 1.0, 5.0, 50.0])  # Pa
t_end = 500.0  # Extended time to see long-term behavior
dt=1.00

results = {}
for sigma in stress_levels:
    t, gamma, gamma_dot, lam = model.simulate_creep(
        sigma_0=float(sigma),
        t_end=t_end,
        dt=dt
    )
    results[float(sigma)] = {
        't': t,
        'gamma': gamma,
        'gamma_dot': gamma_dot,
        'lambda': lam
    }

print(f"Simulated creep at {len(stress_levels)} stress levels")
print(f"Stress range: {stress_levels[0]} - {stress_levels[-1]} Pa")

In [ ]:
# Plot viscosity bifurcation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Strain evolution
for sigma in stress_levels:
    data = results[float(sigma)]
    axes[0, 0].plot(data['t'], data['gamma'], linewidth=2, label=f'σ₀ = {sigma} Pa')

axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Strain γ(t)')
axes[0, 0].set_title('Strain Evolution at Different Stress Levels')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Strain rate evolution
for sigma in stress_levels:
    data = results[float(sigma)]
    axes[0, 1].semilogy(data['t'], jnp.abs(data['gamma_dot']), linewidth=2, label=f'σ₀ = {sigma} Pa')

axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Strain Rate |γ̇(t)| [1/s]')
axes[0, 1].set_title('Strain Rate Evolution (Log Scale)')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Structure evolution
for sigma in stress_levels:
    data = results[float(sigma)]
    axes[1, 0].plot(data['t'], data['lambda'], linewidth=2, label=f'σ₀ = {sigma} Pa')

axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Structure Parameter λ(t)')
axes[1, 0].set_title('Structure Evolution')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()
axes[1, 0].set_ylim([0, 1.1])

# Viscosity evolution (η = η_∞ * (η_0/η_∞)^λ)
eta_inf = model.parameters["eta_inf"].value
eta_0 = model.parameters["eta_0"].value
for sigma in stress_levels:
    data = results[float(sigma)]
    eta = eta_inf * (eta_0 / eta_inf) ** data['lambda']
    axes[1, 1].semilogy(data['t'], eta, linewidth=2, label=f'σ₀ = {sigma} Pa')

axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Viscosity η(t) [Pa·s]')
axes[1, 1].set_title('Viscosity Evolution (Log Scale)')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Phase diagram: steady-state strain rate vs applied stress
# Identify critical yield stress

# Sample more stress levels for phase diagram
stress_scan = jnp.logspace(-1, 2, 30)  # 0.1 to 100 Pa
steady_state_rates = []

for sigma in stress_scan:
    t, gamma, gamma_dot, lam = model.simulate_creep(
        sigma_0=float(sigma),
        t_end=500.0,
        dt=1.0  # Use dt instead of n_points
    )
    # Take final strain rate as proxy for steady state
    steady_state_rates.append(float(jnp.abs(gamma_dot[-1])))

steady_state_rates = jnp.array(steady_state_rates)

# Find critical stress (where rate changes significantly)
# Use threshold: where steady rate > 1e-6 s^-1
yielding_mask = steady_state_rates > 1e-6
if jnp.any(yielding_mask):
    critical_stress_idx = jnp.where(yielding_mask)[0][0]
    critical_stress = stress_scan[critical_stress_idx]
else:
    critical_stress = None

# Plot phase diagram
fig, ax = plt.subplots(figsize=(10, 6))

ax.loglog(stress_scan, steady_state_rates, 'o-', linewidth=2, markersize=6)

if critical_stress is not None:
    ax.axvline(critical_stress, color='red', linestyle='--', linewidth=2, 
               label=f'Critical Stress ≈ {critical_stress:.2f} Pa')

ax.set_xlabel('Applied Stress σ₀ [Pa]')
ax.set_ylabel('Steady-State Strain Rate |γ̇_ss| [1/s]')
ax.set_title('Viscosity Bifurcation: Phase Diagram')
ax.grid(True, alpha=0.3, which='both')
ax.legend()

# Add annotations
if critical_stress is not None:
    ax.text(critical_stress / 3, 1e-8, 'Sub-yield\n(Bounded)', 
            ha='center', va='center', fontsize=11, color='blue',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    ax.text(critical_stress * 3, 1e-2, 'Supra-yield\n(Unbounded)', 
            ha='center', va='center', fontsize=11, color='red',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
display(fig)
plt.close(fig)

if critical_stress is not None:
    print(f"\nEstimated critical yield stress: σ_y ≈ {critical_stress:.2f} Pa")
else:
    print("\nNo clear yield stress identified in this stress range")

## Save Results

In [ ]:
# Create output directory
output_dir = Path("../outputs/dmt/creep")
output_dir.mkdir(parents=True, exist_ok=True)

# Save NLSQ results
nlsq_output = output_dir / "mucus_creep_nlsq.txt"
with open(nlsq_output, 'w') as f:
    f.write("DMT Creep NLSQ Fitting Results\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Applied stress: {sigma_0} Pa\n")
    f.write(f"Data points: {len(time_data)}\n")
    f.write(f"R² score: {metrics['R2']:.6f}\n")
    f.write(f"RMSE: {metrics['RMSE']:.6e}\n\n")
    f.write("Fitted Parameters:\n")
    f.write("-" * 50 + "\n")
    for name, value in fitted_params.items():
        f.write(f"{name:12s}: {value:.6e}\n")

# Save Bayesian results
bayesian_output = output_dir / "mucus_creep_bayesian.txt"
with open(bayesian_output, 'w') as f:
    f.write("DMT Creep Bayesian Inference Results\n")
    f.write("=" * 50 + "\n\n")
    f.write("95% Credible Intervals:\n")
    f.write("-" * 50 + "\n")
    for param_name in param_names:
        if param_name in intervals:
            lower, upper = intervals[param_name]
            median = jnp.median(samples[param_name])
            f.write(f"{param_name:12s}: [{lower:.6e}, {upper:.6e}], median: {median:.6e}\n")

# Save bifurcation data
bifurcation_output = output_dir / "viscosity_bifurcation.csv"
bifurcation_data = np.column_stack([stress_scan, steady_state_rates])
np.savetxt(bifurcation_output, bifurcation_data, delimiter=',', 
           header='Stress[Pa],SteadyStateRate[1/s]', comments='')

print(f"Results saved to: {output_dir}")
print(f"  - NLSQ results: {nlsq_output.name}")
print(f"  - Bayesian results: {bayesian_output.name}")
print(f"  - Bifurcation data: {bifurcation_output.name}")

## Key Takeaways

### Maxwell Creep Mechanics

1. **Elastic Jump**: Instantaneous response γ_e(0) = σ₀/G captures elastic storage
2. **Viscous Flow**: Time-dependent flow γ̇_v = σ₀/η(λ) driven by structure-dependent viscosity
3. **Structure Coupling**: Viscous flow rate feeds back to structure evolution

### Viscosity Bifurcation

1. **Sub-yield** (σ₀ < σ_y): Aging dominates → λ increases → η increases → flow stops
2. **Supra-yield** (σ₀ > σ_y): Rejuvenation dominates → λ decreases → η decreases → continuous flow
3. **Critical Stress**: Marks transition between bounded and unbounded creep

### Delayed Yielding

1. **Near σ_y**: Structure evolution timescale t_eq controls delay to steady state
2. **Biological Relevance**: Mucus under gravity vs applied pressure shows threshold behavior
3. **Thixotropic Signature**: Time-dependent yield transition characteristic of soft materials

### Practical Implications

- **Material Design**: Control t_eq, a, c to tune yielding behavior
- **Processing**: Apply stress above σ_y for continuous flow
- **Quality Control**: Measure creep compliance to infer yield stress and structure timescales